
#  FixMyPhone Pop-up

> **Files**: Upload `about_business.pdf` and `business_summary.txt` in the *Files* pane (or keep them at the default paths below).






In [1]:
!pip install --upgrade --quiet openai==1.* gradio==4.* pypdf==4.* python-dotenv==1.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
google-genai 1.44.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-adk 1.16.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.


In [2]:
import os
import json
from typing import Dict, Any, List

from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr
from openai import OpenAI

# --- Set your key here -------------------------------------------
OPENAI_API_KEY: str = "Your_API"
# ---------------------------------------------------------------------------

if not OPENAI_API_KEY:
    load_dotenv()  # loads variables from a local .env if present
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

if not OPENAI_API_KEY:
    try:
        from getpass import getpass
        OPENAI_API_KEY = getpass("Enter OPENAI_API_KEY (input hidden): ").strip()
    except Exception:
        pass

if not OPENAI_API_KEY:
    raise ValueError(
        "OPENAI_API_KEY not set. Either assign it to the OPENAI_API_KEY variable at the top "
        "of this cell, define it in a .env file, set it in the environment, or enter it when prompted."
    )

client = OpenAI(api_key=OPENAI_API_KEY)
print("OpenAI client initialized.")


OpenAI client initialized.


In [3]:
# Default paths
SUMMARY_TXT_PATH = "business_summary.txt"
ABOUT_PDF_PATH = "about_business.pdf"

# If you're running this directly from the provided files in this environment,
# you can also point here:
alt_txt = "/mnt/data/business_summary.txt"
alt_pdf = "/mnt/data/about_business.pdf"

if os.path.exists(alt_txt):
    SUMMARY_TXT_PATH = alt_txt
if os.path.exists(alt_pdf):
    ABOUT_PDF_PATH = alt_pdf

def read_txt(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return ""

def read_pdf_text(path: str) -> str:
    if not os.path.exists(path):
        return ""
    text_parts = []
    try:
        reader = PdfReader(path)
        for page in reader.pages:
            text_parts.append(page.extract_text() or "")
    except Exception as e:
        print(f"PDF read error: {e}")
    return "\n".join(text_parts)

business_summary = read_txt(SUMMARY_TXT_PATH)
about_business_text = read_pdf_text(ABOUT_PDF_PATH)

print("Summary chars:", len(business_summary))
print("PDF chars    :", len(about_business_text))


Summary chars: 794
PDF chars    : 2264


In [4]:

SYSTEM_PROMPT = f"""
You are FixMyPhone Pop-Up — a friendly mobile device repair kiosk network assistant.
Stay strictly in character and answer using the provided business docs.

PRIMARY SOURCES (verbatim content available to you):
--- business_summary.txt ---
{business_summary}

--- about_business.pdf (OCR) ---
{about_business_text}
---

CORE BEHAVIOR
- Be concise, helpful, and transparent.
- Use only the above documents for facts about the business.
- Collect leads politely: name, email, device model/issue, preferred kiosk, notes.
- If you are unsure or a part seems rare/out-of-catalog, DO NOT guess.
  Call the tool: record_feedback(question) and offer alternatives (ETA, referral, diagnostics).
- Encourage leaving contact info for follow-up.
- Boundaries: no device content access; no bypasses; defer to official OEM policies.

TOOLS YOU CAN CALL
1) record_customer_interest(email, name, message)
   Use when user requests a quote/booking or shares interest.
2) record_feedback(question)
   Use when you are uncertain or the question requires human follow-up.

When calling tools, supply all fields. After a tool runs, continue assisting the user.

STYLE
- Friendly, professional tone.
- One-paragraph answers unless lists improve clarity.
"""
print(SYSTEM_PROMPT[:800])



You are FixMyPhone Pop-Up — a friendly mobile device repair kiosk network assistant.
Stay strictly in character and answer using the provided business docs.

PRIMARY SOURCES (verbatim content available to you):
--- business_summary.txt ---
FixMyPhone Pop-Up — Business Summary

What we do
AI-assisted mobile device repair kiosks in malls and transit hubs. The agent quotes repairs by device model/issue, captures leads (device + email + preferred kiosk), and logs specialist questions for human follow-up.

Mission
Make everyday repairs fast, transparent, and trustworthy—combining skilled techs, ESD-safe processes, and a helpful agent—while protecting customer data and devices.

Services (high level)
• Lead capture (device, email, preferred kiosk).
• Unknown-Q guardrail for specialist questions


In [5]:
import csv
from datetime import datetime

LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
LEADS_CSV = os.path.join(LOG_DIR, "leads.csv")
FEEDBACK_CSV = os.path.join(LOG_DIR, "feedback.csv")

# Initialize CSVs with headers if they don't exist
if not os.path.exists(LEADS_CSV):
    with open(LEADS_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["ts_iso", "email", "name", "message"])

if not os.path.exists(FEEDBACK_CSV):
    with open(FEEDBACK_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["ts_iso", "question"])

def record_customer_interest(email: str, name: str, message: str) -> str:
    ts = datetime.utcnow().isoformat()
    with open(LEADS_CSV, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([ts, email, name, message])
    print(f"[LEAD] {ts} | {name} <{email}>: {message}")
    return f"Lead recorded for {name} <{email}> at {ts}."

def record_feedback(question: str) -> str:
    ts = datetime.utcnow().isoformat()
    with open(FEEDBACK_CSV, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([ts, question])
    print(f"[FEEDBACK] {ts} | {question}")
    return f"Thanks! I've logged this for a human specialist to follow up. Ref ts: {ts}."


In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Log a potential customer lead (name, email, message/notes).",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "Customer email"},
                    "name": {"type": "string", "description": "Customer name"},
                    "message": {"type": "string", "description": "Lead notes / device & issue & preferred kiosk"}
                },
                "required": ["email", "name", "message"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log an unknown/uncertain question to trigger human follow-up.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "The user's question requiring review"}
                },
                "required": ["question"]
            },
        },
    },
]

def call_openai(messages: List[Dict[str, Any]]) -> Dict[str, Any]:
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.3,
        max_tokens=600,
    )
    return resp.model_dump()

def run_agent(user_msg: str, history: List[Dict[str, str]]) -> str:
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for h in history:
        messages.append({"role": "user", "content": h["user"]})
        messages.append({"role": "assistant", "content": h["assistant"]})
    messages.append({"role": "user", "content": user_msg})

    response = call_openai(messages)
    choice = response["choices"][0]
    msg = choice["message"]

    tool_outputs = []
    if "tool_calls" in msg and msg["tool_calls"]:
        for tool_call in msg["tool_calls"]:
            fn_name = tool_call["function"]["name"]
            args = json.loads(tool_call["function"]["arguments"] or "{}")

            if fn_name == "record_customer_interest":
                out = record_customer_interest(**args)
            elif fn_name == "record_feedback":
                out = record_feedback(**args)
            else:
                out = f"Unknown tool: {fn_name}"
            tool_outputs.append({"tool_call_id": tool_call["id"], "name": fn_name, "content": out})

        messages.append({
            "role": "assistant",
            "tool_calls": msg["tool_calls"],
            "content": None
        })
        for to in tool_outputs:
            messages.append({
                "role": "tool",
                "tool_call_id": to["tool_call_id"],
                "name": to["name"],
                "content": to["content"],
            })
        response2 = call_openai(messages)
        final_msg = response2["choices"][0]["message"]["content"]
        return final_msg

    return msg.get("content", "")

In [7]:
import gradio as gr

with gr.Blocks(fill_height=True, theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📱 FixMyPhone Pop-Up — Business Agent")
    gr.Markdown("Ask about services, warranty, pricing approach, or leave your device+issue to get a quote.")

    # history: List[Tuple[user:str, assistant:str]]
    def chat_fn(message, history):
        internal_history = []
        for u, a in history:
            internal_history.append({"user": u or "", "assistant": a or ""})
        reply = run_agent(message, internal_history)  # assumes run_agent is defined elsewhere
        return reply

    gr.ChatInterface(
        fn=chat_fn,
        # default type='chatbot' -> history is list of tuples
        examples=[
            "Hi! Do you access my data during repairs?",
            "I have a cracked iPhone 13 screen. Can I get a quote? Preferred kiosk: Downtown Mall.",
            "Do you carry rare foldable hinge parts for the Galaxy Z Fold2?",
        ],
        title="FixMyPhone Pop-Up Assistant",
        autofocus=True,
        retry_btn=None,
        undo_btn=None,
        submit_btn="Send",
        clear_btn="Clear",
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ed826307b21ce2db42.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:

if OPENAI_API_KEY:
    print("Quick test:", run_agent("my iphone 11 is not charging properly", []))
else:
    print("Set OPENAI_API_KEY to run quick test.")

Quick test: I can help you with that! Could you please provide me with your name, email address, and the preferred kiosk location where you'd like to book a repair? Additionally, if there are any specific details about the charging issue, please share those as well.
